# Example usage

To use `nf_parser` in a project:

In [ ]:
import nf_parser

print(nf_parser.__version__)

In [103]:
test_1 = """#!/usr/bin/env nextflow

nextflow.enable.dsl=2

include { foo } from './module/1.nf'
include { BAR } from './module/2.nf'

//def sayHello() {
//    println "$params.foo $params.bar"
//}

/*
 * Defines the pipeline input parameters (with a default value for each one).
 * Each of the following parameters can be specified as command line options.
 */
params.in = "$baseDir/data/sample.fa" // path to fa file
// some comment
params.query = "$baseDir/data/sample.fa"
params.db = "$baseDir/blast-db/pdb/tiny"
params.out = "result.txt"
params.chunkSize = 100

process foo {
    container 'python'

    output:
    path 'sample.txt'

    shell:
    '''
    echo 'hello' >  sample.txt
    '''
}

workflow  {
    foo()
}

"""

In [106]:
## # https://chat.openai.com/share/fb9d0650-3f66-4104-bff3-8697d5aeaf2e

from pathlib import Path
from lark import Lark

test_file = test_1
#nextflow_grammar = Path("../nf_parser/grammer/nf.lark").read_text()
nextflow_grammar = Path("./grammers/nf.lark").read_text()
parser = Lark(nextflow_grammar, parser='lalr')
tree = parser.parse(test_file)
print(tree)
print(tree.pretty())

Tree(Token('RULE', 'start'), [Tree(Token('RULE', 'shebang'), [Token('SH_COMMENT', '#!/usr/bin/env nextflow')]), Tree(Token('RULE', 'dsl'), [Token('NUMBER', '2')]), Tree(Token('RULE', 'module'), [Token('CNAME', 'foo'), Token('STRING', "'./module/1.nf'")]), Tree(Token('RULE', 'module'), [Token('CNAME', 'BAR'), Token('STRING', "'./module/2.nf'")]), Tree(Token('RULE', 'comment'), [Token('CPP_COMMENT', '//def sayHello() {')]), Tree(Token('RULE', 'comment'), [Token('CPP_COMMENT', '//    println "$params.foo $params.bar"')]), Tree(Token('RULE', 'comment'), [Token('CPP_COMMENT', '//}')]), Tree(Token('RULE', 'comment'), [Token('C_COMMENT', '/*\n * Defines the pipeline input parameters (with a default value for each one).\n * Each of the following parameters can be specified as command line options.\n */')]), Tree(Token('RULE', 'param'), [Token('CNAME', 'in'), Tree('string', [Token('DQUOTED_STRING', '"$baseDir/data/sample.fa"')]), Tree(Token('RULE', 'comment'), [Token('CPP_COMMENT', '// path to 